In [95]:
import pandas as pd
pd.set_option('display.max_columns', 200)
import numpy as np
from datetime import timedelta, datetime 
import plotly.express as px

In [96]:
fecha_corte = '2022-11-04'
dt_string = datetime.now().strftime('%y%m%d')

In [97]:
fc = '221104'
fc_file = '221104'
res = pd.read_excel(f'output/{fc}/{fc_file}_resultado_recobro.xlsx', sheet_name='F4', dtype=str)
res.columns = [col.upper() for col in res.columns]
res['FECHA_RESERVA'] = pd.to_datetime(res['FECHA_RESERVA'], format='%Y-%m-%d')
res['FECHA REGISTR.DIARIO'] = pd.to_datetime(res['FECHA REGISTR.DIARIO'], format='%Y-%m-%d')
res['TOTAL_COSTO'] = pd.to_numeric(res['TOTAL_COSTO'])
res['mes_f4'] = res['FECHA_RESERVA'].dt.strftime('%b')
res['mes_sap'] = res['FECHA REGISTR.DIARIO'].dt.strftime('%b-%y')
res['IMPORTE (MON.SOC.)'] = pd.to_numeric(res['IMPORTE (MON.SOC.)'])
#res['costo_total_f11_cd'] = pd.to_numeric(res['costo_total_f11_cd'])
res['ENVIADO_CONTABILIDAD'] = res['ENVIADO_CONTABILIDAD'].str.strip()
res['DOC_CONTABLE'] = res['DOC_CONTABLE'].str.strip()

In [98]:
res.loc[res['ENVIADO_CONTABILIDAD'] =='F4 DE MERMA']

,LOCAL,FECHA_RESERVA,CTECH_KEY,NFOLIO,TRANSPORTADORA_F4,DOC_CONTABLE,REF_SAP,ENVIADO_CONTABILIDAD,FECHA REGISTR.DIARIO,ASIENTO COMPENSACIÓN,AC CREADO POR,TRANSPORTADORA_SAP,TP.ASIENTO CONTABLE,INDICADOR SAP,IMPORTE (MON.SOC.),TOTAL_COSTO,mes_f4,mes_sap
4561,9903 - CD FALABELLA,2022-11-01 10:19:16,46921833,NaN,Tcc,F4 DE MERMA,F4 DE MERMA,F4 DE MERMA,NaT,NaN,NaN,NaN,NaN,No encontrado en SAP,0,139900,Nov,NaN


In [99]:
f11 = pd.read_csv('C:/Users/maperezr/OneDrive - Falabella/General/DATA/F11/221010_f11.csv', sep=';', dtype='object') 
f11['FECHA_CREACION'] = pd.to_datetime(f11['FECHA_CREACION'], format='%Y-%m-%d')
f11['TOTAL_COSTO'] = pd.to_numeric(f11['TOTAL_COSTO'])
gb_f11 = f11.groupby(['NRO_F11', 'FECHA_CREACION', 'PROPIETARIO', 'ESTADO', 'SERVICIO', 'GRUPO'])['TOTAL_COSTO'].sum().reset_index()

gb_f11.columns=['F11_' + col.upper() for col in gb_f11.columns]
gb_f11.rename(columns={'F11_NRO_F11':'NFOLIO'}, inplace=True)

print(f'La base de f11 tiene {f11.shape} registros con {gb_f11.shape} f11s unicos, ')

La base de f11 tiene (84180, 39) registros con (70743, 7) f11s unicos, 


In [100]:
gb_f11.head()

,NFOLIO,F11_FECHA_CREACION,F11_PROPIETARIO,F11_ESTADO,F11_SERVICIO,F11_GRUPO,F11_TOTAL_COSTO
0,1111187383,2021-10-10,EMPRESA,Entrega total,S.TECNICO,TIENDAS,1775551.262
1,1111187390,2021-10-10,EMPRESA,Entrega total,S.TECNICO,TIENDAS,713290.766
2,1111187406,2021-10-10,EMPRESA,Entrega total,S.TECNICO,TIENDAS,633048.943
3,1111187697,2021-10-11,EMPRESA,Entrega total,S.TECNICO,TIENDAS,732866.000
4,1111187703,2021-10-11,EMPRESA,Entrega total,RETIRO F12,CD,54498.009


In [101]:
gb_f11.loc[gb_f11.duplicated('NFOLIO', keep=False)]

,NFOLIO,F11_FECHA_CREACION,F11_PROPIETARIO,F11_ESTADO,F11_SERVICIO,F11_GRUPO,F11_TOTAL_COSTO
5024,1111259424,2021-11-13,CLIENTE,Despachado,RETIRO F12,CD,55410.000
5025,1111259424,2021-11-13,EMPRESA,Despachado,RETIRO F12,CD,82922.445
14776,1111381705,2021-12-23,CLIENTE,Despachado,RETIRO F12,CD,33313.590
14777,1111381705,2021-12-23,EMPRESA,Despachado,RETIRO F12,CD,59219.638
48206,1111783103,2022-07-07,CLIENTE,Despachado,RETIRO F12,CD,109350.000
48207,1111783103,2022-07-07,EMPRESA,Despachado,RETIRO F12,CD,115470.000


In [102]:
completo = res.merge(gb_f11, how='left', on='NFOLIO')

In [103]:
completo.loc[completo['ENVIADO_CONTABILIDAD']=='F4 DE MERMA']

,LOCAL,FECHA_RESERVA,CTECH_KEY,NFOLIO,TRANSPORTADORA_F4,DOC_CONTABLE,REF_SAP,ENVIADO_CONTABILIDAD,FECHA REGISTR.DIARIO,ASIENTO COMPENSACIÓN,AC CREADO POR,TRANSPORTADORA_SAP,TP.ASIENTO CONTABLE,INDICADOR SAP,IMPORTE (MON.SOC.),TOTAL_COSTO,mes_f4,mes_sap,F11_FECHA_CREACION,F11_PROPIETARIO,F11_ESTADO,F11_SERVICIO,F11_GRUPO,F11_TOTAL_COSTO
4561,9903 - CD FALABELLA,2022-11-01 10:19:16,46921833,NaN,Tcc,F4 DE MERMA,F4 DE MERMA,F4 DE MERMA,NaT,NaN,NaN,NaN,NaN,No encontrado en SAP,0,139900,Nov,NaN,NaT,NaN,NaN,NaN,NaN,NaN


In [104]:
dc_var = 'DOC_CONTABLE'
sap_var = 'INDICADOR SAP'
f11_fcreacion = 'F11_FECHA_CREACION'
transportes_status = 'ENVIADO_CONTABILIDAD'
freserva_f4 = 'FECHA_RESERVA'
fregistro_sap = 'FECHA REGISTR.DIARIO'
tc = 'TOTAL_COSTO'
completo.loc[completo[dc_var]=='NOTA CREDITO', dc_var] = 'NC'
completo.loc[completo[dc_var]=='RECIBO DE EXONERACIÓN', dc_var] = 'RECIBO DE EXONERACION'
completo.loc[completo[dc_var]=='REXO', dc_var] = 'RECIBO DE EXONERACION'

completo.loc[(completo[dc_var]=='FELE') & (completo[sap_var]=='Encontrado en SAP'), 'comentario gco'] = 'FELE encontrada en SAP'
completo.loc[(completo[dc_var]=='FELE') & (completo[sap_var]=='No encontrado en SAP'), 'comentario gco'] = 'FELE no encontrada en SAP'

completo.loc[(completo[dc_var]=='RECIBO DE EXONERACION') & (completo[sap_var]=='Encontrado en SAP'), 'comentario gco'] = 'REXO encontrado en SAP'
completo.loc[(completo[dc_var]=='RECIBO DE EXONERACION') & (completo[sap_var]=='No encontrado en SAP'), 'comentario gco'] = 'REXO no encontrado en SAP'

completo.loc[(completo[dc_var]=='NC') & (completo[sap_var]=='Encontrado en SAP'), 'comentario gco'] = 'NC encontrada en SAP'
completo.loc[(completo[dc_var]=='NC') & (completo[sap_var]=='No encontrado en SAP'), 'comentario gco'] = 'NC no encontrada en SAP'

completo.loc[(completo[dc_var]=='CXC') & (completo[sap_var]=='Encontrado en SAP'), 'comentario gco'] = 'CXC encontrado en SAP'
completo.loc[(completo[dc_var]=='CXC') & (completo[sap_var]=='No encontrado en SAP'), 'comentario gco'] = 'CXC no encontrado en SAP'

completo.loc[completo[f11_fcreacion].isna() & completo[transportes_status].isna() & (completo[freserva_f4]<=pd.to_datetime(fecha_corte, format='%Y-%m-%d')), 'comentario gco']='No presentes en base de CD'
completo.loc[completo[f11_fcreacion].isna() & completo[transportes_status].isna() & (completo[freserva_f4]>pd.to_datetime(fecha_corte, format='%Y-%m-%d')), 'comentario gco']='F4 realizados después de fecha de corte'


# Verificar posible falsa alarma porque base de angel solo 1 año dinamico 
#cond_1 = completo['f11_fecha_creacion'].isna() & ~completo['comentario gco'].isin(['F4 realizados después de fecha de corte','No presentes en base de CD'])
#completo.loc[cond_1, 'comentario gco'] = completo.loc[cond_1, 'comentario gco'] + ' + F11 presenta novedad'

cond_2 = (completo[freserva_f4]<=pd.to_datetime('2022-03-31', format='%Y-%m-%d')) & ((completo[sap_var]=='No encontrado en SAP'))
completo.loc[cond_2, 'comentario gco'] = completo.loc[cond_2, 'comentario gco'] + ' + F4 con fecha reserva mayor a 60 días'

cond_3= (completo[fregistro_sap]) < (completo[freserva_f4]-timedelta(days=30))
completo.loc[cond_3, 'comentario gco'] =  completo.loc[cond_3, 'comentario gco'] + ' + Registro SAP anterior a F4'
completo.loc[cond_3, 'sap_ant_f4'] = 'y'

completo.loc[(completo[transportes_status]=='PENDIENTE'), 'comentario gco'] = 'Docuemento contable pendiente'

In [105]:
completo.loc[(completo[transportes_status]=='F4 DE MERMA'), 'comentario gco']

4561    NaN
Name: comentario gco, dtype: object

In [106]:
completo.groupby('comentario gco')[tc].sum().sort_values(ascending=False)

comentario gco
FELE encontrada en SAP                                              1215621235
CXC encontrado en SAP                                                723877399
NC encontrada en SAP                                                 410320086
FELE no encontrada en SAP                                            350694159
REXO encontrado en SAP                                               262286901
NC no encontrada en SAP                                              113244295
REXO no encontrado en SAP                                             77387072
CXC no encontrado en SAP                                              64902702
NC encontrada en SAP + Registro SAP anterior a F4                     37198579
FELE no encontrada en SAP + F4 con fecha reserva mayor a 60 días      17157265
CXC encontrado en SAP + Registro SAP anterior a F4                    14809983
NC no encontrada en SAP + F4 con fecha reserva mayor a 60 días         5736084
Name: TOTAL_COSTO, dtype: int64

In [107]:
completo.loc[completo['comentario gco'].isna()][transportes_status].unique()

array(['F4 DE MERMA', nan], dtype=object)

In [108]:
completo.to_excel(f'{dt_string}_recobro.xlsx', sheet_name='DB', index=False)

In [109]:
completo.columns

Index(['LOCAL', 'FECHA_RESERVA', 'CTECH_KEY', 'NFOLIO', 'TRANSPORTADORA_F4',
       'DOC_CONTABLE', 'REF_SAP', 'ENVIADO_CONTABILIDAD',
       'FECHA REGISTR.DIARIO', 'ASIENTO COMPENSACIÓN', 'AC CREADO POR',
       'TRANSPORTADORA_SAP', 'TP.ASIENTO CONTABLE', 'INDICADOR SAP',
       'IMPORTE (MON.SOC.)', 'TOTAL_COSTO', 'mes_f4', 'mes_sap',
       'F11_FECHA_CREACION', 'F11_PROPIETARIO', 'F11_ESTADO', 'F11_SERVICIO',
       'F11_GRUPO', 'F11_TOTAL_COSTO', 'comentario gco', 'sap_ant_f4'],
      dtype='object')

## grafiacs

aux = completo.groupby(['REF_SAP','importe (mon.soc.)','mes_sap', 'mes_f4']).agg({'tp.asiento contable':'count', 'total_costo':'sum'}).reset_index()
aux_2 = completo.groupby(['ref_sap','importe (mon.soc.)','mes_sap']).agg({'tp.asiento contable':'count', 'total_costo':'sum'}).reset_index()

aux_2.groupby(['mes_sap']).agg({'importe (mon.soc.)':'sum'}).reset_index()

In [110]:
comentarios_ok = ['FELE encontrada en SAP', 'NC encontrada en SAP', 'REXO encontrado en SAP', 'CXC encontrado en SAP', 'NC encontrada en SAP + Registro SAP anterior a F4']
completo.loc[~completo['comentario gco'].isin(comentarios_ok), 'mes_sap']= 'Pendiente recobro'

In [111]:
def set_columns_sum(base, var, column):    
    lista = base.loc[base[var].notna()][var].unique()
    gb_var = base.groupby(var)[column].sum().reset_index()
    gb_var.set_index(var, inplace=True)
    for item in lista:
        base.loc[base[var]==item, var] = f'{item} ${round(gb_var.loc[item, column]/1e6):,.0f}M'
    return base 

In [112]:
mes_x_mes = completo.groupby(['mes_sap', 'mes_f4']).agg({'TP.ASIENTO CONTABLE':'count', 'TOTAL_COSTO':'sum'}).reset_index()
mes_x_mes = set_columns_sum(mes_x_mes, 'mes_f4','TOTAL_COSTO')

In [113]:
dif = completo.groupby(['REF_SAP','IMPORTE (MON.SOC.)']).agg({'TP.ASIENTO CONTABLE':'count', 'TOTAL_COSTO':'sum'}).reset_index()
dif['saldo'] = dif['IMPORTE (MON.SOC.)'] - dif['TOTAL_COSTO'] 

In [123]:
table = pd.pivot_table(mes_x_mes, values='TOTAL_COSTO', index=['mes_f4'],
                    columns=['mes_sap'], aggfunc=np.sum)#, fill_value= 0 ,margins = True, margins_name = 'Total' )

In [124]:
table.applymap(lambda x: f"$ {x/1e6:,.0f}M" if x != np.nan  else 0)

mes_sap,Apr-22,Aug-22,Dec-21,Feb-22,Jan-22,Jul-22,Jun-22,Mar-22,May-22,Nov-21,Oct-21,Oct-22,Pendiente recobro,Sep-21,Sep-22
mes_f4,,,,,,,,,,,,,,,
Apr $405M,$ 124M,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ 1M,$ 273M,$ nanM,$ nanM,$ nanM,$ 7M,$ nanM,$ nanM
Aug $584M,$ nanM,$ 444M,$ 1M,$ nanM,$ nanM,$ 0M,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ 97M,$ 0M,$ 41M
Feb $579M,$ 160M,$ nanM,$ 5M,$ 26M,$ 0M,$ nanM,$ 21M,$ 315M,$ 40M,$ nanM,$ nanM,$ nanM,$ 12M,$ nanM,$ nanM
Jan $31M,$ 5M,$ 0M,$ 1M,$ nanM,$ 10M,$ nanM,$ nanM,$ 8M,$ nanM,$ nanM,$ nanM,$ nanM,$ 6M,$ nanM,$ nanM
Jul $189M,$ 0M,$ 13M,$ 0M,$ 0M,$ nanM,$ 134M,$ 5M,$ nanM,$ 0M,$ nanM,$ nanM,$ nanM,$ 37M,$ 0M,$ nanM
Jun $233M,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ 233M,$ 0M,$ 0M,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM
Mar $216M,$ 15M,$ 1M,$ 10M,$ 15M,$ 1M,$ nanM,$ 57M,$ 107M,$ 5M,$ 0M,$ 0M,$ nanM,$ 4M,$ nanM,$ nanM
May $294M,$ 5M,$ nanM,$ nanM,$ nanM,$ nanM,$ 45M,$ 87M,$ nanM,$ 147M,$ 0M,$ 2M,$ nanM,$ 3M,$ 5M,$ nanM
Nov $377M,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ 377M,$ nanM,$ nanM


In [125]:
table

mes_sap,Apr-22,Aug-22,Dec-21,Feb-22,Jan-22,Jul-22,Jun-22,Mar-22,May-22,Nov-21,Oct-21,Oct-22,Pendiente recobro,Sep-21,Sep-22
mes_f4,,,,,,,,,,,,,,,
Apr $405M,124275092.0,NaN,NaN,NaN,NaN,NaN,NaN,504218.0,272793971.0,NaN,NaN,NaN,6935827.0,NaN,NaN
Aug $584M,NaN,444031674.0,892552.0,NaN,NaN,360236.0,NaN,NaN,NaN,NaN,NaN,NaN,97157586.0,58760.0,41315893.0
Feb $579M,160455093.0,NaN,4931863.0,25994790.0,129900.0,NaN,20648883.0,314610377.0,40115028.0,NaN,NaN,NaN,12405208.0,NaN,NaN
Jan $31M,5222061.0,372295.0,858091.0,NaN,10483741.0,NaN,NaN,8445635.0,NaN,NaN,NaN,NaN,6017672.0,NaN,NaN
Jul $189M,108224.0,12825762.0,230000.0,135259.0,NaN,133821324.0,4799698.0,NaN,92834.0,NaN,NaN,NaN,37081414.0,55550.0,NaN
Jun $233M,NaN,NaN,NaN,NaN,NaN,NaN,232690717.0,255800.0,232276.0,NaN,NaN,NaN,NaN,NaN,NaN
Mar $216M,15244995.0,1192236.0,10404544.0,14525348.0,759846.0,NaN,56781006.0,106768712.0,5435102.0,156597.0,6464.0,NaN,4470469.0,NaN,NaN
May $294M,4978945.0,NaN,NaN,NaN,NaN,45063918.0,86636438.0,NaN,146772288.0,216362.0,2249100.0,NaN,2848119.0,5063532.0,NaN
Nov $377M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,376683343.0,NaN,NaN


In [126]:
table = table.reindex(['Jan $31M','Feb $579M', 'Mar $216M', 'Apr $405M', 'May $294M', 'Jun $233M', 'Jul $189M', 'Aug $584M', 'Sep $172M','Oct $214M', 'Nov $377M', 'Total'], columns=['Sep-21','Oct-21', 'Nov-21','Dec-21', 'Jan-22', 'Feb-22', 'Mar-22', 'Apr-22', 'May-22','Jun-22', 'Jul-22' ,'Aug-22', 'Sep-22', 'Oct-22', 'Pendiente recobro', 'Total'])

In [127]:
import numpy as np 
table.applymap(lambda x: f"$ {x/1e6:,.0f}M")

mes_sap,Sep-21,Oct-21,Nov-21,Dec-21,Jan-22,Feb-22,Mar-22,Apr-22,May-22,Jun-22,Jul-22,Aug-22,Sep-22,Oct-22,Pendiente recobro,Total
mes_f4,,,,,,,,,,,,,,,,
Jan $31M,$ nanM,$ nanM,$ nanM,$ 1M,$ 10M,$ nanM,$ 8M,$ 5M,$ nanM,$ nanM,$ nanM,$ 0M,$ nanM,$ nanM,$ 6M,$ nanM
Feb $579M,$ nanM,$ nanM,$ nanM,$ 5M,$ 0M,$ 26M,$ 315M,$ 160M,$ 40M,$ 21M,$ nanM,$ nanM,$ nanM,$ nanM,$ 12M,$ nanM
Mar $216M,$ nanM,$ 0M,$ 0M,$ 10M,$ 1M,$ 15M,$ 107M,$ 15M,$ 5M,$ 57M,$ nanM,$ 1M,$ nanM,$ nanM,$ 4M,$ nanM
Apr $405M,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ 1M,$ 124M,$ 273M,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ 7M,$ nanM
May $294M,$ 5M,$ 2M,$ 0M,$ nanM,$ nanM,$ nanM,$ nanM,$ 5M,$ 147M,$ 87M,$ 45M,$ nanM,$ nanM,$ nanM,$ 3M,$ nanM
Jun $233M,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ 0M,$ nanM,$ 0M,$ 233M,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM
Jul $189M,$ 0M,$ nanM,$ nanM,$ 0M,$ nanM,$ 0M,$ nanM,$ 0M,$ 0M,$ 5M,$ 134M,$ 13M,$ nanM,$ nanM,$ 37M,$ nanM
Aug $584M,$ 0M,$ nanM,$ nanM,$ 1M,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ 0M,$ 444M,$ 41M,$ nanM,$ 97M,$ nanM
Sep $172M,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ 9M,$ 149M,$ nanM,$ 14M,$ nanM


In [128]:
table

mes_sap,Sep-21,Oct-21,Nov-21,Dec-21,Jan-22,Feb-22,Mar-22,Apr-22,May-22,Jun-22,Jul-22,Aug-22,Sep-22,Oct-22,Pendiente recobro,Total
mes_f4,,,,,,,,,,,,,,,,
Jan $31M,NaN,NaN,NaN,858091.0,10483741.0,NaN,8445635.0,5222061.0,NaN,NaN,NaN,372295.0,NaN,NaN,6017672.0,NaN
Feb $579M,NaN,NaN,NaN,4931863.0,129900.0,25994790.0,314610377.0,160455093.0,40115028.0,20648883.0,NaN,NaN,NaN,NaN,12405208.0,NaN
Mar $216M,NaN,6464.0,156597.0,10404544.0,759846.0,14525348.0,106768712.0,15244995.0,5435102.0,56781006.0,NaN,1192236.0,NaN,NaN,4470469.0,NaN
Apr $405M,NaN,NaN,NaN,NaN,NaN,NaN,504218.0,124275092.0,272793971.0,NaN,NaN,NaN,NaN,NaN,6935827.0,NaN
May $294M,5063532.0,2249100.0,216362.0,NaN,NaN,NaN,NaN,4978945.0,146772288.0,86636438.0,45063918.0,NaN,NaN,NaN,2848119.0,NaN
Jun $233M,NaN,NaN,NaN,NaN,NaN,NaN,255800.0,NaN,232276.0,232690717.0,NaN,NaN,NaN,NaN,NaN,NaN
Jul $189M,55550.0,NaN,NaN,230000.0,NaN,135259.0,NaN,108224.0,92834.0,4799698.0,133821324.0,12825762.0,NaN,NaN,37081414.0,NaN
Aug $584M,58760.0,NaN,NaN,892552.0,NaN,NaN,NaN,NaN,NaN,NaN,360236.0,444031674.0,41315893.0,NaN,97157586.0,NaN
Sep $172M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8979921.0,148913743.0,NaN,14129191.0,NaN


In [129]:
import plotly.express as px
data=[[1, 25, 30, 50, 1], [20, 1, 60, 80, 30], [30, 60, 1, 5, 20]]
fig = px.imshow(table, text_auto = '.2s', title='Mes de reserva F4 vs mes de registro en SAP', labels={'x':'Mes SAP', 'y':'Mes reserva F4'})
fig.update_xaxes(side="top")
fig.write_image(f"{dt_string}_mes_x_mes.png", height=500,width=1000, engine='orca')
fig.show()

In [130]:
tabla = pd.pivot_table(mes_x_mes, values='TOTAL_COSTO', index=['mes_f4'],
                    columns=['mes_sap'], aggfunc=np.sum, margins = True, margins_name='Total')
tabla = tabla.reindex(['Jan $34M','Feb $620M', 'Mar $262M','Apr $517M','May $294M', 'Jun $233M', 'Jul $189M', 'Aug $584M','Total'], columns=[ 'Dec', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug','Pendiente recobro', 'Total'],  )
tabla

mes_sap,Dec,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Pendiente recobro,Total
mes_f4,,,,,,,,,,,
Jan $34M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Feb $620M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mar $262M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apr $517M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
May $294M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2848119.0,2.938287e+08
Jun $233M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.331788e+08
Jul $189M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37081414.0,1.891501e+08
Aug $584M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97157586.0,5.838167e+08
Total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,644071460.0,3.293376e+09


In [131]:
import numpy as np 
tabla.applymap(lambda x: f"$ {x/1e6:,.0f}M" if x != np.nan else 0)

mes_sap,Dec,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Pendiente recobro,Total
mes_f4,,,,,,,,,,,
Jan $34M,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM
Feb $620M,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM
Mar $262M,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM
Apr $517M,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM
May $294M,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ 3M,$ 294M
Jun $233M,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ 233M
Jul $189M,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ 37M,$ 189M
Aug $584M,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ 97M,$ 584M
Total,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ nanM,$ 644M,"$ 3,293M"


## recobro j11

df = pd.read_excel('220510_recobro_j11.xlsx', sheet_name='F4 Detalle')

df.groupby(['linea'])['total_costo_f4'].sum()

j11 = df.loc[df['linea']=='j11'].reset_index(drop=True)

j11['enviado a contabilidad'].unique()

pendiente =['PENDIENTE GENERAR NC ', 'PENDIENTE GENERAR OC', 'PENDIENTE SOLICTUD BONIFICACION CONTABILIDAD ']

j11.loc[j11['enviado a contabilidad'].isin(pendiente), 'estado_recobro'] = 'Pendiente gestión CD'

j11.loc[j11['enviado a contabilidad']=='OK ENVIADO A CONTABILIDAD', 'estado_recobro'] = 'Gestión CD finalizada'

j11.loc[j11['enviado a contabilidad']==0, 'estado_recobro'] = 'Diferencia entre SRX y CD'

j11.loc[j11['enviado a contabilidad']=='CD cambia motivo de F4 de recobro a merma', 'estado_recobro'] = 'CD cambia motivo de F4 de recobro a merma'

j11.groupby(['estado_recobro'])['total_costo_f4'].sum().sort_values(ascending=False)

j11.groupby(['comentario_gco'])['total_costo_f4'].sum().sort_values(ascending=False)

j11['comentario_gco'].unique()

espera = ['No presentes en base de CD + F4 con fecha reserva mayor a 60 días',
       'REXO no encontrado en SAP + F4 con fecha reserva mayor a 60 días',
       'NC no encontrada en SAP + F4 con fecha reserva mayor a 60 días',
       'FELE no encontrada en SAP + F4 con fecha reserva mayor a 60 días',
       'NC encontrada en SAP + Registro SAP anterior a F4',
       'F4 realizados después de fecha de corte']

j11.loc[j11['comentario_gco'].isin(espera), 'mes_sap']= 'En espera'

j11.columns = [col.lower() for col in j11.columns]

import numpy as np

j11.loc[j11['fecha registr.diario'] == '2000-01-01', 'fecha registr.diario'] = np.NaN
j11['mes_f4'] = j11['fecha_reserva'].dt.strftime('%b')
j11['mes_sap'] = j11['fecha registr.diario'].dt.strftime('%b')
j11.loc[j11['mes_sap'].isna(), 'mes_sap']= 'En espera'

mes_x_mes = j11.groupby(['mes_sap', 'mes_f4']).agg({'tp.asiento contable':'count', 'total_costo_f4':'sum'}).reset_index()
mes_x_mes = set_columns_sum(mes_x_mes, 'mes_f4','total_costo_f4')
mes_x_mes = set_columns_sum(mes_x_mes, 'mes_sap','total_costo_f4')
mes_x_mes

table = pd.pivot_table(mes_x_mes, values='total_costo_f4', index=['mes_f4'],
                    columns=['mes_sap'], aggfunc=np.sum)

table.applymap(lambda x: f"$ {x:,.0f}")

table = table.reindex(['Jan $24M','Feb $496M', 'Mar $202M', 'Apr $415M'], columns=[ 'Feb $24M', 'Mar $369M', 'Apr $105M','En espera $640M'])

import plotly.express as px
data=[[1, 25, 30, 50, 1], [20, 1, 60, 80, 30], [30, 60, 1, 5, 20]]
fig = px.imshow(table, text_auto = '.2s',  labels={'x':'Mes SAP', 'y':'Mes reserva F4'})
fig.update_xaxes(side="top")
fig.write_image(f"f4_j11_recobro.svg",scale=1, height=500,width=700, engine='orca')
fig.show()

import plotly.express as px
data=[[1, 25, 30, 50, 1], [20, 1, 60, 80, 30], [30, 60, 1, 5, 20]]
fig = px.imshow(table, text_auto = '.2s', title='Mes de reserva F4 vs mes de registro en SAP', labels={'x':'Mes SAP', 'y':'Mes reserva F4'})
fig.update_xaxes(side="top")
fig.write_image(f"f4_j11_recobro.svg",scale=1, height=500,width=900, engine='orca')
fig.show()

tabla = pd.pivot_table(mes_x_mes, values='total_costo_f4', index=['mes_f4'],
                    columns=['mes_sap'], aggfunc=np.sum, margins = True, margins_name='Total')
tabla = tabla.reindex(['Jan $24M','Feb $496M', 'Mar $202M','Apr $415M','Total'], columns=[ 'Feb $24M', 'Mar $369M', 'Apr $105M','En espera $640M', 'Total'])
tabla

tabla.applymap(lambda x: f"$ {x:,.0f}")

j11.to_excel('220512_f4_recobro_j11.xlsx')